In [1]:
import tensorflow as tf
import numpy as np
import os
from models.utils import brand_new_tfsession, draw_tree
from tensorflow.keras.callbacks import EarlyStopping, Callback
from models import ConvNet, SoftBinaryDecisionTree
from tensorflow.keras.models import load_model

sess = brand_new_tfsession()

In [2]:
# 检查是否启用了急切执行模式
if tf.executing_eagerly():
    print("Eager execution is enabled.")
else:
    print("Eager execution is not enabled.")

Eager execution is not enabled.


In [3]:
# 设置环境变量来控制 TensorFlow 日志级别 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # 只显示错误信息 
# TensorFlow 2.x 中设置日志级别 
tf.get_logger().setLevel('ERROR') 

In [4]:
#上采样
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import *
# from keras.layers import *
import tensorflow as tf
import numpy as np
from sklearn import preprocessing
import xlwt

def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

original_data = pd.read_csv('./dataset/AEEEM/EQ.csv')                

original_data.isnull().values.any()  # Gives false ie:No null value in dataset
original_data = original_data.fillna(value=False)  #将缺失值填充为False
original_Y = original_data['class']  # Defective   class   isDefective  defects
original_Y = pd.DataFrame(original_Y) 
original_data = normalization(original_data) 




#  将数据写入新文件  
#original_data.to_excel('C:/Users/lenovo/Desktop/excel/ar6.xls',sheet_name='ar6',index=False)
original_X = pd.DataFrame(original_data.drop(['class'], axis=1))  #Defective   class  isDefective  defects
#print(original_X)
#分为训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(original_X, original_Y, test_size=.1, random_state=12)
# now we resample, and from that we take training and validation sets 
sm = SMOTE(random_state=12, sampling_strategy=1.0) # 解决分类不平衡问题 
x, y = sm.fit_resample(x_train, y_train)
y_train = pd.DataFrame(y, columns=['class']) #Defective  class  isDefective  defects 
x_train = pd.DataFrame(x, columns=original_X.columns)
#print(x_test) 

#细分出验证集
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.1, random_state=12)

x_train = x_train.values
x_val = x_val.values
x_test = x_test.values
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

img_rows, img_cols =  7,3    # 7,3   13,2
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

(88, 13, 2, 1) (88, 1) (10, 13, 2, 1) (10, 1) (7, 13, 2, 1) (7, 1)


In [5]:
img_rows, img_cols, img_chans, n_classes =  7,3  , 1, 2  #7,3   13,2
print(img_rows, img_cols, img_chans, n_classes)

13 2 1 2


In [6]:
# 将标签转换为独热编码向量
y_train = tf.keras.utils.to_categorical(y_train, n_classes)
y_val = tf.keras.utils.to_categorical(y_val, n_classes)
y_test = tf.keras.utils.to_categorical(y_test, n_classes)

print(y_train.shape, y_val.shape, y_test.shape)
# print(y_train)

(88, 2) (10, 2) (7, 2)


In [7]:
# 将输入数据进行标准化并转换成浮点型
x_train = (x_train / np.max(x_train)).astype(np.float32)
x_val = (x_val / np.max(x_val)).astype(np.float32)
x_test = (x_test / np.max(x_test)).astype(np.float32)

In [8]:
# 首先将数据展平，变成二维数据
x_train_flat = x_train.reshape((x_train.shape[0], -1))
x_val_flat = x_val.reshape((x_val.shape[0], -1))
x_test_flat = x_test.reshape((x_test.shape[0], -1))

x_train_flat.shape, x_val_flat.shape, x_test_flat.shape

((88, 26), (10, 26), (7, 26))

In [9]:
# 定义超参数
n_features = img_rows * img_cols * img_chans
tree_depth = 4
penalty_strength = 1e+1     
penalty_decay = 0.25     
ema_win_size = 1000    
inv_temp = 0.01  
learning_rate = 1e-06  
batch_size = 2   

In [10]:
sess = brand_new_tfsession(sess)

# 构建g模型
g_model_joint = SoftBinaryDecisionTree(tree_depth, n_features, n_classes, 
    penalty_strength=penalty_strength, penalty_decay=penalty_decay, 
    inv_temp=inv_temp, ema_win_size=ema_win_size, learning_rate=learning_rate)
g_model_joint.build_model()
g_model_joint.initialize_variables(sess, x_train_flat, batch_size)

# 构建f模型
f_model_joint = ConvNet(img_rows, img_cols, img_chans, n_classes)   
f_model_joint.build_model()  


Built tree has 16 leaves out of 31 nodes


<tf.Tensor 'dense_1/BiasAdd:0' shape=(None, 2) dtype=float32>

In [11]:
def analyze(f, g, x_test, x_test_flat, y_test):
    # 初始化混淆矩阵
    conf_matx_fy = np.zeros([2, 2])  # 对于f模型和真实标签y的混淆矩阵
    conf_matx_fg = np.zeros([2, 2])  # 对于f模型和g模型的混淆矩阵，用于计算保真度
    conf_matx_gy = np.zeros([2, 2])  # 对于g模型和真实标签y的混淆矩阵，用于计算g模型的准确度

    # 对整个数据集进行预测
    output_f = f.predict(x_test)
    output_g = g.predict(x_test_flat)
    y = np.argmax(y_test, axis=1)
    pred_f = np.argmax(output_f, axis=1)
    pred_g = np.argmax(output_g, axis=1)

    # 更新混淆矩阵
    for i in range(len(y)):
        conf_matx_fy[y[i], pred_f[i]] += 1
        conf_matx_fg[pred_f[i], pred_g[i]] += 1
        conf_matx_gy[y[i], pred_g[i]] += 1

    # 计算准确度和保真度
    accuracy_f = np.diag(conf_matx_fy).sum() / len(y) 
    fidelity = np.diag(conf_matx_fg).sum() / len(y)  
    accuracy_g = np.diag(conf_matx_gy).sum() / len(y)  

    return accuracy_f, fidelity, accuracy_g

In [12]:
import os
from joint import train, f_model_evaluate, g_model_evaluate


In [13]:
from tqdm import tqdm 
import os 
 
epochs = 40 
batch_size = 64 
learning_rate_train = 1e-04
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_train)  
# 设置模型保存的路径   
f_model_path = 'assets/f_model_joint.hdf5'  
g_model_path = 'assets/g_model_joint'   
                                       
# 检查模型文件是否存在  
f_model_exists = os.path.exists(f_model_path)  
g_model_exists = os.path.exists(g_model_path + '.index')  

# 如果模型文件不存在，则训练模型   
if not f_model_exists or not g_model_exists:   
    f_model_joint, g_model_joint = train(f_model_joint, g_model_joint, sess, x_train,  
                                         x_train_flat, y_train, optimizer, epochs, batch_size=batch_size)  
    # 保存 f_model_jonit 
    f_model_joint.model.save(f_model_path)   
    print("Model saved to %s" % f_model_path)   

    # 保存 g_model_joint    
    g_model_joint.saver.save(sess, g_model_path)  
    print("Model saved to %s" % g_model_path)  
else:
    # 加载 f_model_jonit 
    f_model_joint.model = load_model(f_model_path)  
    print("f_model_joint successfully loaded from %s" % f_model_path)  

    # 加载 g_model_joint 
    g_model_joint.saver.restore(sess, g_model_path) 
    print("g_model_joint successfully restored from %s" % g_model_path) 

# 进行模型性能评估  
f_model_joint.evaluate(x_test, y_test) 
g_model_joint.evaluate(x=x_test_flat, y=y_test, batch_size=2) 

f_model_joint successfully loaded from assets/f_model_joint.hdf5
g_model_joint successfully restored from assets/g_model_joint
accuracy: 85.71% | loss: 0.6894818544387817
accuracy: 71.43% | loss: inf


In [14]:
# 进行模型评估
print("f_model_joint:")
f_model_joint.evaluate(x_test, y_test)

print("g_model_joint:")
g_model_joint.evaluate(x=x_test_flat, y=y_test, batch_size=2)

f_model_joint:
accuracy: 85.71% | loss: 0.6894818544387817
g_model_joint:
accuracy: 71.43% | loss: 36.634307861328125


In [15]:
# 计算保真度
f_joint_acc, fidelity, g_joint_acc = analyze(f_model_joint, g_model_joint, x_test, x_test_flat, y_test)

print("Accuracy of f (in %): {:.2f}".format(f_joint_acc * 100))
print("Accuracy of g (in %): {:.2f}".format(g_joint_acc * 100))
print("Fidelity (in %): {:.2f}".format(fidelity * 100))


Accuracy of f (in %): 85.71
Accuracy of g (in %): 71.43
Fidelity (in %): 85.71
